In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys
import glob
from os import listdir
from music21 import *
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#Keras Imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.recurrent import LSTM

Using Theano backend.


In [172]:
#PianoRoll
sys.path.append(os.getcwd()+"/../../PianoRoll/")
sys.path.append(os.getcwd()+"/../../")
from PianoRoll import PianoRoll
from Utils import Utils
from metrics import Metrics

In [103]:
res_factor = 12
note_threshold = 0.1
#Composer paths
chord_dir = os.getcwd() + "/../../data/testData/chords/"
composition_dir = os.getcwd() + "/compositions/"
#Checkpoint paths
model_dir = os.getcwd() + "/checkpoints/model/"
weights_dir = os.getcwd() + "/checkpoints/weights/"

#Other constants
MIDI_FILE_EXTENSION = "mid"
NOTE_ON = "note_on"
NOTE_OFF = "note_off"
MAX_NOTE = "max_note"
MIN_NOTE = "min_note"
TICKS = "ticks"
MIN_NOTE_INITIALIZER = 10000
MAX_NOTE_INITIALIZER = 0
MAX_TICK_INITIALIZER = 0
MELODY_LOWEST_NOTE = 60
DEFAULT_TICKS_PER_BEAT = 96

In [ ]:
chord_files = glob.glob("%s*.mid" %(chord_dir))
composition_files = []
for i in range(len(chord_files)):
    composition_files.append('generated_%d' %(i+1))
print(chord_files, composition_files)

In [ ]:
chord_test_roll = PianoRoll(chord_dir, res_factor=res_factor)
print("# of Ticks:{}\tMax_Note:{}\tMin_Note:{}".format(chord_test_roll.ticks,
                                                  chord_test_roll.max_note,
                                                  chord_test_roll.min_note))

In [ ]:
chord_test_data = chord_test_roll.generate_piano_roll_func()
doubled_chords = np.tile(chord_test_data, (1, 2, 1))
X_test = PianoRoll.create_test_inputs(doubled_chords, chord_test_roll.ticks)
print(X_test.shape)

In [ ]:
# #Load Model
# model_files = listdir(model_dir)
# print("Choose a file for the model:")
# print("---------------------------------------")
# for i, file in enumerate(model_files):
#     print(str(i) + " : " + file)
# print("---------------------------------------")
# file_index = input("Enter the index of the file model that you want:\t")
# model_path = model_dir + model_files[file_index]
# print("Loading model from {}".format(model_path))
# model = model_from_json(open(model_path).read())

In [ ]:
model = Sequential()

In [ ]:
from keras.layers.recurrent import LSTM

In [ ]:
model.add(LSTM(input_dim=12, output_dim=24, activation='sigmoid', return_sequences=False))
# model.add(LSTM(output_dim=18,activation="sigmoid", return_sequences=True))
# model.add(LSTM(output_dim=24, activation="sigmoid", return_sequences=False))
# model.add(LSTM(output_dim=24, activation="sigmoid", return_sequences=True))
# model.add(LSTM(output_dim=24, activation="sigmoid", return_sequences=False))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
model.load_weights("checkpoints/weights/1layer_24_6000epochs.h5")

In [ ]:
print(X_test.shape)

In [ ]:
!pip install music21

In [ ]:
parsed_file = converter.parse("../../data/trainData/melody/001.mid")

In [ ]:
parsed_file.show()

In [ ]:
gen_stream = converter.parse("../../data/testData/melody_composition/gen_001.mid")
gen_stream.show()

In [ ]:
parsed_file.plot()

In [ ]:
score.plot()

In [ ]:
tst_roll = PianoRoll.fromMidiCreatePianoRoll([os.getcwd()+"/../../data/testData/melody_composition/001.mid"], 256, 67, 24, 12)

In [ ]:
tst_roll = np.squeeze(tst_roll)

In [ ]:
max_pitches = np.argmax(tst_roll, axis=1)
max_pitches[max_pitches!=0] += (67 - 1)

In [ ]:
from itertools import groupby
pitch_with_duration = [(key, len(list(group))) for key, group in groupby(max_pitches)]

In [ ]:
s = stream.Stream()
for i in range(0, len(pitch_with_duration)):
    pitch_val, dur_count = pitch_with_duration[i]
    if(pitch_val == 0):
        n = note.Rest()
        n.duration.quarterLength = (0.125 * dur_count)
    else:
        p = pitch.Pitch()
        p.midi = pitch_val
        n = note.Note(p.midi, quarterLength = (0.125 * dur_count))
    s.append(n)
# print(s.pitches)
print(s.duration.quarterLength)

In [ ]:
score = stream.Stream()
part = stream.Part()
# part.clef = clef.BassClef()
# part.append(instrument.Harpsichord())
part.insert(s)
score.insert(part)

In [ ]:
mf = midi.translate.streamToMidiFile(score)
mf.open(os.getcwd() + '/../../data/testData/melody_composition/gen_001.mid', 'wb')
mf.write()
mf.close()  

In [137]:
def chord2roll_wrapper(dir_path, num_ticks, lowest_note, num_cols, res_factor):
    chord_files = glob.glob("%s*.mid" %(chord_dir))
    piano_roll = np.zeros((len(chord_files), num_ticks, num_cols))
    for chord_file_id in range(len(chord_files)):
        roll = chord2roll(chord_files[chord_file_id], num_ticks, lowest_note, num_cols, res_factor)
        piano_roll[chord_file_id, :, :] = roll
    return piano_roll

In [161]:
def chord2roll(chord_file, num_ticks, lowest_note, num_cols, res_factor):
    cstream = converter.parse(chord_file)
    part = cstream.elements[0]
    timesteps_so_far = 0
    ticks_per_beat = DEFAULT_TICKS_PER_BEAT/res_factor
    roll = np.zeros((num_ticks, num_cols))
    for nr in part.notesAndRests:
        if isinstance(nr, note.Rest):
            c_pitches = [0]
        else:
            c_pitches = [p.midi - lowest_note + 1 for p in nr.pitches]
        start = timesteps_so_far
        dur_count = float(nr.quarterLength)
        timesteps_so_far += int(ticks_per_beat * dur_count)
        end = timesteps_so_far
        roll[start:end,c_pitches] = 1.0
    return roll

In [164]:
chord_roll = chord2roll("../../data/trainData/chords/016.mid", 256, 48, 13, 12)
chord_roll[0:38]

[0]
[3, 6, 10]
[2, 5, 8, 10]
[1, 6, 10]
[3, 6, 10]
[2, 5, 8, 10]
[1, 6, 10]


array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0., 

In [89]:
DEFAULT_TICKS_PER_BEAT = 96
def midi2roll(path_to_file, num_ticks, lowest_note, num_cols, res_factor=12):
    ticks_per_beat = DEFAULT_TICKS_PER_BEAT / res_factor  # Integer division
    midi_stream = converter.parse(path_to_file)
    roll = np.zeros((num_ticks, num_cols))
    timesteps_so_far = 0
    for nr in midi_stream.elements[0].notesAndRests:
        print(nr)
        p = nr.pitch.midi if isinstance(nr, note.Note) else 0
        duration = float(nr.quarterLength)
        col_idx = (p - lowest_note) + 1 if p != 0 else p  # Adding to distinguish the case where there is no note.
        start = timesteps_so_far
        timesteps_so_far += int(ticks_per_beat * duration)
        end = timesteps_so_far
        roll[start:end, col_idx] = 1.0
    return roll

In [165]:
def midi2roll_wrapper(midi_dir, num_ticks, lowest_note, num_cols, keys_count = {},res_factor=12):
    midi_files = Utils.get_matching_files(midi_dir+"*.mid")
    piano_roll = np.zeros((len(midi_files), num_ticks, num_cols))
    for i in range(len(midi_files)):
        midi_file = midi_files[i]
        piano_roll[i, :, :], keys_count = midi2roll(midi_file, 
                                                    num_ticks, 
                                                    lowest_note, 
                                                    num_cols, 
                                                    keys_count ,
                                                    res_factor)
    return piano_roll

In [173]:
melody_roll = PianoRoll.midi2roll_wrapper(os.getcwd() + "/../../data/trainData/melody/", 256, 60, 25,res_factor=12)

AttributeError: type object 'PianoRoll' has no attribute 'midi2roll_wrapper'

In [16]:
chord_roll = midi2roll_wrapper(os.getcwd() + "/../../data/trainData/chords/", 256, 48, 13, 12)
print(chord_roll.shape)

(68, 256, 13)


In [23]:
chord_doubles = np.tile(chord_roll, (1, 2, 1))
melody_doubles = np.tile(melody_roll, (1, 2, 1))
X_train, Y_train = PianoRoll.createNetInputs(chord_doubles, melody_doubles, seq_length=256)
print(X_train.shape, Y_train.shape)

(68, 512, 13)
((17408, 256, 13), (17408, 25))


In [143]:
chord_files = glob.glob("%s*.mid" %(chord_dir))
composition_files = []
for i in range(len(chord_files)):
    composition_files.append(composition_dir+'generated_%d.mid' %(i+1))
print(chord_files, composition_files)

(['/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/1_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/2_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/3_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/4_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/5_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../data/testData/chords/6_testchord.mid', '/Users/arulselvanmadhavan/cs6140-machine-learning/Keras_projects/LSTM-RNN-Melody-Composer/models/lstm/../../d

In [76]:
def roll2midi_wrapper(save_dir, mel_roll, num_ticks, lowest_note, res_factor):
    quarter_note_resolution = 1./(DEFAULT_TICKS_PER_BEAT / res_factor)
    for roll_id in range(mel_roll.shape[0]):
        roll = mel_roll[0]
        max_pitches = np.argmax(roll, axis=1)
        max_pitches[max_pitches != 0] += lowest_note
        pitch_with_duration = [(key, len(list(group))) for key, group in groupby(max_pitches)]
        s = stream.Stream()
        for i in range(0, len(pitch_with_duration)):
            pitch_val, dur_count = pitch_with_duration[i]
            if(pitch_val == 0):
                n = note.Rest()
                n.duration.quarterLength = (quarter_note_resolution * dur_count)
            else:
                p = pitch.Pitch()
                p.midi = pitch_val
                n = note.Note(p.midi, quarterLength = (quarter_note_resolution * dur_count))
            s.append(n)
        score = stream.Stream()
        part = stream.Part()
        # part.clef = clef.BassClef()
        # part.append(instrument.Harpsichord())
        part.insert(s)
        score.insert(part)
        mf = midi.translate.streamToMidiFile(score)
        mf.open(composition_files[roll_id], 'wb')
        mf.write()
        mf.close()

In [144]:
roll2midi_wrapper(composition_files, chord_roll[0,:,:] , 256, 48, 12)

In [59]:
1./8

0.125

In [54]:
s = stream.Stream()
for i in range(0, len(pitch_with_duration)):
    pitch_val, dur_count = pitch_with_duration[i]
    if(pitch_val == 0):
        n = note.Rest()
        n.duration.quarterLength = (0.125 * dur_count)
    else:
        p = pitch.Pitch()
        p.midi = pitch_val
        n = note.Note(p.midi, quarterLength = (0.125 * dur_count))
    s.append(n)
# print(s.pitches)
# print(s.duration.quarterLength)

In [55]:
score = stream.Stream()
part = stream.Part()
# part.clef = clef.BassClef()
# part.append(instrument.Harpsichord())
part.insert(s)
score.insert(part)

In [56]:
mf = midi.translate.streamToMidiFile(score)
mf.open(os.getcwd() + '/../../data/testData/melody_composition/gen_001.mid', 'wb')
mf.write()
mf.close()

In [ ]:
path_to_file = os.getcwd()+"/../../data/testData/melody_composition/001.mid"
num_ticks = 256
lowest_melody_note = 67
num_cols = 24
res_factor = 12

# def piano_roll_music21(path_to_file, num_ticks, lowest_melody_note, num_cols, res_factor):
midi_stream = converter.parse(path_to_file)
roll = np.zeros((num_ticks, num_cols))

timesteps_so_far = 0
for nr in midi_stream.elements[0].notesAndRests:
    p = nr.pitch.midi  if isinstance(nr, note.Note) else 0
    duration = float(nr.quarterLength)
    col_idx = (p - lowest_melody_note) + 1 if p !=0 else p #Adding to distinguish the case where there is no note.
    start = timesteps_so_far
    timesteps_so_far += int(8 * duration)
    end = timesteps_so_far
    roll[start:end, col_idx] = 1.0
    
# for i in range(len(midi_stream.pitches)):
#     p = midi_stream.pitches[i]
#     col_idx = (p.midi - lowest_melody_note) + 1 #Adding to distinguish the case where there is no note.
#     print("{}\t{}".format(p.midi, col_idx))
# #     print(len(midi_stream.pitches))
# #     print(midi_stream.quarterLength)